# Michelson tutorial
## Chapter four  

Among all operations that you can execute in a smart contract, arithmetic operations are probably the most common ones, but also the simplest ones. After all, computers in general were created to provide an easy way to do arithmetic operations that started to become too complex.  

Within your smart contract, you may want to add or subtract tokens to the balance of your users or you may want to multiply or divide different values. As you will understand it when reading what follows, it will be extremely easy in Michelson. You only have two conditions to remember before trying any operation: first, you must make sure that you have two elements on top of the stack. Second, you must verify that these two elements are of comparable types and of types that work together.  

Unlike programming languages that you may already be familiar with, Michelson doesn't use arithmetic operators you know like **+**, **-**, __*__ or **/**. Everything is "instruction-based" and you will have to use instructions that modify the stack to make operations.

1. Available types for arithmetic operations
2. The **`ADD`** instruction
3. The **`SUB`** instruction
4. The **`MUL`** instruction
5. The **`EDIV`** instruction
6. Other operations on numeric values

##### 1. Available types for arithmetic operations

As you may have already guessed, Michelson allows us to work with `int` and `nat` for arithmetic operations. An operation of two `int` will yield an `int` and an operation of two`nat` will yield a `nat`. These two types can be used together for addition, subtraction, multiplication and division but as long there is an `int` in your operation, the result will also be an `int`. For example, `int + nat = int`.  

In addition of `int` and `nat`, you can use other types for arithmetic operations, for example `mutez` and `timestamp`. Here is a table that sums up all the possible combinations and the type of the result:  

| Value type 	| Available operation 	| Value type 	| Result type 	|
|:----------:	|:-------------------:	|:----------:	|:-----------:	|
|     int    	|   ADD/SUB/MUL/EDIV  	|     int    	|     int     	|
|     nat    	|   ADD/SUB/MUL/EDIV  	|     nat    	|     nat     	|
|     int    	|   ADD/SUB/MUL/EDIV  	|     nat    	|     int     	|
|  timestamp 	|       ADD/SUB       	|     int    	|  timestamp  	|
|  timestamp 	|         SUB         	|  timestamp 	|     int     	|
|    mutez   	|       ADD/SUB       	|    mutez   	|    mutez    	|
|    mutez   	|       MUL/EDIV      	|     nat    	|    mutez    	|

A few considerations to keep in mind regarding the table above:  
- Be always very mindful about the return type when you put `int` and `nat` together. You may spend some time scratching your head and wondering why you are not getting the type you are expecting because you overlooked the types you are using in your operation.
- A few operations are available for types outside of `int` and `nat`, but they are somewhow limited (for logical reasons). For example, it wouldn't make any sense to multiply timestamps with other values or multiply `mutez` together.
- Other limitations are set in place for safety purposes: limiting the possible operations on `mutez` prevent negative balances or integer overflow.

##### 2. The **`ADD`** instruction

The first instruction we are going to play with is the **`ADD`** instruction. As its name suggests, it takes two values and add them together. If you refer to the table above, you can observe that it is one of the most widely available instruction throughout the different types. **`ADD`** also allows you to use different types of values, keeping in mind that the return type is always fix.  

Let's check some examples and see how it works:

In [16]:
## We quickly initialize a contract environment to manipulate different values on the stack
storage unit ;
parameter unit ;
BEGIN Unit Unit ;
DROP ;

storage unit;
parameter unit;
BEGIN: use %default; drop all; push (Unit, Unit);
DROP: pop (Unit, Unit);

In this first example, we push 2 `int` on to the stack and add them.  
As you can see, the return value is of type `int` and is the result of `5 + 5`.

In [17]:
PUSH int 5 ;
PUSH int 5 ;
ADD ;

PUSH: push 5;
PUSH: push 5;
ADD: pop 5, 5; push 10;

value,type
10,int


Now we have a value of type `int` on top of the stack. Let's push a value of type `nat` and see what happens:

In [18]:
PUSH nat 5 ;
ADD ;

PUSH: push 5;
ADD: pop 5, 10; push 15;

value,type
15,int


As expected, `int 10 + nat 5` equals `int 15`. The addition of `int` and `nat` always yields an `int` (*for logical reasons, the result of something like `int -20 + nat 5` cannot be a `nat` value*).  

In the next snippet, you will see a new instruction, **`NOW`**. We will come back to it in a later chapter, just know for now that it pushes the current timestamp on top of the stack. We can then use the timestamp to demonstrate how **`ADD`** works with values of this type:

In [19]:
NOW ;
ADD ;

NOW: push 1592033776;
ADD: pop 1592033776, 15; push 1592033791;

value,type
1592033791,timestamp


According to the day and time you are running this code block, the timestamp will be different. However, you should clearly see that the initial value has been incremented with `15`, which was the value we had in our stack when we pushed the timestamp. You can also push a value and add it to the existing timestamp:

In [13]:
PUSH int 500 ;
ADD ;

PUSH: push 500;
ADD: pop 500, 1592032504; push 1592033004;

value,type
1592033004,timestamp


Let's clean our stack and see how adding `mutez` together works. As in the previous examples, we just push two values on top of the stack and add them. This yields a result in `mutez`:

In [14]:
DROP ;
PUSH mutez 50 ;
PUSH mutez 20 ;
ADD ;

DROP: pop 1592033004;
PUSH: push 50;
PUSH: push 20;
ADD: pop 20, 50; push 70;

value,type
70,mutez


Keep in mind that `mutez` values are NOT tezzies but a *representation* of their value. When writing `PUSH mutez 50`, you are not manipulating tezzies, only their representation. There are specific instructions we will study later that tell the contract to send actual tezzies.